## Test Connection and get sample data

#### Use Curl to check if your API key works for some instruments

The curl can be found in kaiko [documentation](https://docs.kaiko.com/kaiko-rest-api/data-feeds/reference-data/derivatives-reference) 


In [38]:
!curl --compressed -H "Accept: application/json" -H "X-Api-Key: $KAIKO_API_KEY" \
"https://us.market-api.kaiko.io/v2/data/derivatives.v2/reference?exchange=drbt&instrument_class=option&base_assets=btc&quote_assets=usd&start_time=2024-12-05T00:00:00.000Z" | jq


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  8222    0  8222    0     0   3943      0 --:--:--  0:00:02 --:--:--  3945
{
  "query": {
    "exchange": "drbt",
    "instrument_class": "option",
    "base_assets": [
      "btc"
    ],
    "quote_assets": [
      "usd"
    ],
    "start_time": "2024-12-05T00:00:00.000Z",
    "data_version": "v2",
    "commodity": "derivatives",
    "request_time": "2024-12-19T11:05:27.369Z"
  },
  "time": "2024-12-19T11:05:29.072Z",
  "timestamp": 1734606329072,
  "data": [
    {
      "exchange": "drbt",
      "instrument_class": "option",
      "instrument": "btc27dec2410000c",
      "base": "btc",
      "quote": "usd",
      "contract_size": "1",
      "contract_size_unit": "btc",
      "listing_timestamp": "2023-12-28 08:00:09 UTC",
      "expiry": "2024-12-27 08:00:00 UTC",
      "strike_price": "10000",
      "underlying_index": "BTC-

#### Use Python requests to check if your API key works for some instruments

In [24]:
import os
import requests
import pandas as pd

kaiko_api_key = os.environ['KAIKO_API_KEY']

def get_derivatives_metrics(url_derivatives_metrics):
    headers = {
        'Accept': 'application/json',
        'X-Api-Key': kaiko_api_key,
    }

    response = requests.get(url_derivatives_metrics, headers=headers)

    res = response.json()
    res_data = res['data']

    # Loop for pagination, does not retry if there is an error
    while True:
        if (res.get('next_url') is not None) & (res['data'] != []):
            response = requests.get(res['next_url'], headers=headers)
            res = response.json()
            res_data = res_data + res['data']
        else:
            break

    try:
        res_data = pd.DataFrame(res_data)
        if 'timestamp' in res_data.columns:
            res_data['timestamp'] = pd.to_datetime(res_data['timestamp'], unit='ms')
        return res_data
    except KeyError:
        return pd.DataFrame()

In [27]:
exch = 'drbt'
base_assets = 'btc'
quote_assets = 'usd'
instrument_class = 'option'

url_derivatives_reference = (f"https://us.market-api.kaiko.io/v2/data/derivatives.v2/reference"
                             f"?exchange={exch}"
                             f"&instrument_class={instrument_class}"
                             f"&base_assets={base_assets}"
                             f"&quote_assets={quote_assets}"
                             # f"&start_time={start_time}"  # if using start_time or end_time, don't use page_size
                             # f"&end_time={end_time}"
                             f"&page_size=1000")
df_ref = get_derivatives_metrics(url_derivatives_metrics=url_derivatives_reference)
df_ref

,exchange,instrument_class,instrument,base,quote,contract_size,contract_size_unit,listing_timestamp,expiry,strike_price,underlying_index
0,drbt,option,btc10apr204750c,btc,usd,1,btc,2020-03-28 03:21:00 UTC,2020-04-10 08:00:00 UTC,4750,SYN.BTC-10APR20
1,drbt,option,btc10apr204750p,btc,usd,1,btc,2020-03-28 03:21:00 UTC,2020-04-10 08:00:00 UTC,4750,SYN.BTC-10APR20
2,drbt,option,btc10apr205000c,btc,usd,1,btc,2020-03-26 08:19:00 UTC,2020-04-10 08:00:00 UTC,5000,SYN.BTC-10APR20
3,drbt,option,btc10apr205000p,btc,usd,1,btc,2020-03-26 08:19:00 UTC,2020-04-10 08:00:00 UTC,5000,SYN.BTC-10APR20
4,drbt,option,btc10apr205250c,btc,usd,1,btc,2020-03-26 08:00:08 UTC,2020-04-10 08:00:00 UTC,5250,SYN.BTC-10APR20
...,...,...,...,...,...,...,...,...,...,...,...
89059,drbt,option,btc9sep2464000p,btc,usd,1,btc,2024-09-06 08:01:01 UTC,2024-09-09 08:00:00 UTC,64000,SYN.BTC-9SEP24
89060,drbt,option,btc9sep2465000c,btc,usd,1,btc,2024-09-06 08:01:01 UTC,2024-09-09 08:00:00 UTC,65000,SYN.BTC-9SEP24
89061,drbt,option,btc9sep2465000p,btc,usd,1,btc,2024-09-06 08:01:01 UTC,2024-09-09 08:00:00 UTC,65000,SYN.BTC-9SEP24
89062,drbt,option,btc9sep2466000c,btc,usd,1,btc,2024-09-06 12:34:00 UTC,2024-09-09 08:00:00 UTC,66000,SYN.BTC-9SEP24


In [50]:
interval = '1d'

# ----- First example - on Option -----
# exch = 'drbt'
# instrument_class = 'option'
# instrument = "btc27dec24108000c"  # This info can be found in derivatives reference result for options

# ----- Second example - on Perp -----
start_time = '2024-12-09T00:00:00.000Z'
end_time = '2024-12-10T00:00:00.000Z'
exch = 'binc'
instrument_class = 'perpetual-future'
instrument = "btc-usdt"

url_derivatives_risk = ("https://us.market-api.kaiko.io/v2/data/derivatives.v2/risk"
                            f"?exchange={exch}"
                            f"&instrument_class={instrument_class}"
                            f"&instrument={instrument}"
                            f"&page_size=1000"
                            # f"&start_time={start_time}"
                            # f"&end_time={end_time}"
                            f"&interval={interval}")

df_risk = get_derivatives_metrics(url_derivatives_metrics=url_derivatives_risk)
df_risk

KeyboardInterrupt: 

In [41]:
interval = '1m'
instrument = "btc-usdt"
instrument_class = 'perpetual-future'

url_derivatives_price = ("https://us.market-api.kaiko.io/v2/data/derivatives.v2/price"
                         f"?exchange={exch}"
                         f"&instrument_class={instrument_class}"
                         f"&instrument={instrument}"
                         "&page_size=1000"
                         f"&start_time={start_time}"
                         f"&end_time={end_time}")

df_price = get_derivatives_metrics(url_derivatives_metrics=url_derivatives_price)
df_price

,timestamp,index_price,mark_price,price,settlement_price,settlement_timestamp
0,2024-12-09 23:59:00,97348.9893,97402.9481,97427.0,None,None
1,2024-12-09 23:58:00,97394.8736,97449.3323,97427.0,None,None
2,2024-12-09 23:57:00,97453.3464,97506.1815,97427.0,None,None
3,2024-12-09 23:56:00,97412.965,97470.6984,97427.0,None,None
4,2024-12-09 23:55:00,97411.8986,97464.6853,97427.0,None,None
...,...,...,...,...,...,...
1435,2024-12-09 00:04:00,100963.5836,101013.6217,100610.0,None,None
1436,2024-12-09 00:03:00,100846.7786,100888.2538,100610.0,None,None
1437,2024-12-09 00:02:00,100971.595,101016.6306,100610.0,None,None
1438,2024-12-09 00:01:00,101110.54,101161.9542,100610.0,None,None
